In [2]:
import pandas as pd
import random
import dask.dataframe as dd

In [3]:
# Load the data
sample_df = dd.read_csv("../data/100_15_2021.csv.gz").compute()
sample_df.head()

/home/soo/git_repos/datahackdays_2024/.venv/lib/python3.10/site-packages/dask/dataframe/io/csv.py:555: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


,id,timestamp,value_kwh,year
0,f75148515d279e2c6c36207fa4002faf,2021-05-08T00:15:00.000Z,0.0,2021
1,f75148515d279e2c6c36207fa4002faf,2021-01-30T06:30:00.000Z,0.0,2021
2,f75148515d279e2c6c36207fa4002faf,2021-01-25T23:15:00.000Z,0.0,2021
3,f75148515d279e2c6c36207fa4002faf,2021-02-03T18:30:00.000Z,0.0,2021
4,f75148515d279e2c6c36207fa4002faf,2021-06-17T19:15:00.000Z,0.0,2021


In [6]:
# Add month column
sample_df["timestamp"] = pd.to_datetime(sample_df["timestamp"])
sample_df["year_month"] = sample_df["timestamp"].dt.to_period("M")
sample_df.head()

/tmp/ipykernel_65570/135626291.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  sample_df["year_month"] = sample_df["timestamp"].dt.to_period("M")


,id,timestamp,value_kwh,year,year_month
0,f75148515d279e2c6c36207fa4002faf,2021-05-08 00:15:00+00:00,0.0,2021,2021-05
1,f75148515d279e2c6c36207fa4002faf,2021-01-30 06:30:00+00:00,0.0,2021,2021-01
2,f75148515d279e2c6c36207fa4002faf,2021-01-25 23:15:00+00:00,0.0,2021,2021-01
3,f75148515d279e2c6c36207fa4002faf,2021-02-03 18:30:00+00:00,0.0,2021,2021-02
4,f75148515d279e2c6c36207fa4002faf,2021-06-17 19:15:00+00:00,0.0,2021,2021-06


In [7]:
# Step 1: Group by 'id' and extract features
monthly_features_df = sample_df.groupby(["id", "year_month"]).agg(
    {"value_kwh": ["mean", "std", "min", "max"]}
)

monthly_features_df.columns = ["_".join(col).strip() for col in monthly_features_df.columns.values]
monthly_features_df = monthly_features_df.reset_index()
monthly_features_df.head()

,id,year_month,value_kwh_mean,value_kwh_std,value_kwh_min,value_kwh_max
0,035293b7f95ace18fc6c574cc4e6cc74,2021-01,0.050968,0.018580,0.0,0.095
1,035293b7f95ace18fc6c574cc4e6cc74,2021-02,0.046885,0.019188,0.0,0.097
2,035293b7f95ace18fc6c574cc4e6cc74,2021-03,0.045167,0.018714,0.0,0.095
3,035293b7f95ace18fc6c574cc4e6cc74,2021-04,0.042099,0.018532,0.0,0.096
4,035293b7f95ace18fc6c574cc4e6cc74,2021-05,0.037923,0.018846,0.0,0.095


In [8]:
# Save the data as csv
monthly_features_df.to_csv("../data/100_15_2021_features.csv", index=False)